In [3]:
import numpy as np
import pandas as pd

In [4]:
import os
import sys
import glob

In [42]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [6]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import warnings
warnings.filterwarnings("ignore")

In [7]:
DATA_ROOT_DIR = os.path.abspath('../data')
DATABASE_DATA_DIR = os.path.join(DATA_ROOT_DIR,'Database')
TEST_DATA_DIR = os.path.join(DATA_ROOT_DIR,'Test')
OUTPUT_DATA_DIR = os.path.join(DATA_ROOT_DIR,'Output')

files_DATABASE = glob.glob(os.path.join(DATABASE_DATA_DIR, 'Database.xlsx'))

In [8]:
print("Program is starting")
print(" ")

Program is starting
 


# 함수 Definition

In [26]:
#현재 알고리즘의 Dictionary를 출력하는 함수
def TIMES_ROUNDS_dictionary(df):
    temp=df.set_index(['TIMES','ROUNDS'])
    set_temp=list(temp.index.unique())
    dictionary=dict()
    for i in range(len(set_temp)):
                dictionary[i]=set_temp[i]
    return dictionary

In [27]:
#Database의 데이터로 알고리즘을 학습시키는 함수
#입력: database, dictionary
#출력: 학습한 catboost 모델 저장
def catboost_model(df, dictionary):
    
    #예측을 위한 임시적인 새로운 칼럼 생성
    print("모델 생성을 위한 데이터베이스 전처리")
    df['TIMES_ROUNDS']=-1
    for i in range(len(df)):
        for j in range(len(dictionary)):
            if tuple(df[['TIMES','ROUNDS']].iloc[i])==dictionary[j]:
                df.at[i,'TIMES_ROUNDS']=j
                break
    
    #데이터 갯수가 3개 이하인거는 제외하고 학습을 시켜야함(validation에 unique 데이터가 들어가는 것을 방지하기 위해서)
    temp=df.groupby(['TIMES_ROUNDS']).count()

    #데이터 갯수가 3개 이하인 TIMES_ROUNDS 값 리스트
    drop_values=list(temp[temp['MAT_NAME']<=3].index)
    drop_values

    #drop할 인덱스들의 리스트 생성
    drop_list=[]
    for i in range(len(drop_values)):
        drop_list.extend(list(df.loc[(df['TIMES_ROUNDS']==drop_values[i])].index))
    print(drop_list)

    df.drop(drop_list,axis='index',inplace=True)            
                
    
    #train_test_split
    train_set, non_train_set = train_test_split(df, test_size=0.2, random_state=20)
    test_set, valid_set = train_test_split(non_train_set, test_size=0.5, random_state=20)

    y_train = train_set['TIMES_ROUNDS']
    train_set.drop(['TIMES_ROUNDS','TIMES','ROUNDS'], axis=1, inplace=True)
    y_valid = valid_set['TIMES_ROUNDS']
    valid_set.drop(['TIMES_ROUNDS','TIMES','ROUNDS'], axis=1, inplace=True)
    y_test = test_set['TIMES_ROUNDS']
    test_set.drop(['TIMES_ROUNDS','TIMES','ROUNDS'], axis=1, inplace=True)

    #Classifier만들기
    catboost = CatBoostClassifier(iterations=50)

    #훈련데이터에 fitting
    catboost.fit(train_set, y_train, cat_features=['DEV_STYLE_NUMBER','PROCESS','PRT_PART_NAME','MAT_NAME','PRT_NAME','PRT_COLOR','CHEMICALS','GRAD'],
                 eval_set=(valid_set, y_valid))

    #test dataset을 이용한 정확도 평가
    pred=catboost.predict(test_set)
    print('예측 정확도:',accuracy_score(y_test,pred))


    #fitting 된 model을 폴더에 저장
    catboost.save_model(os.path.join(OUTPUT_DATA_DIR,'Model/catboost_TIMES_ROUNDS'),format="cbm")

In [28]:
#Test할 데이터의 기본적인 전처리를 하는 함수
def data_preprocess(df, features):
    
    #컬럼마가 다르게 NULL값 처리

    #(Char)'MAT_NAME','PRT_NAME','PROCESS','PRT_PART_NAME','CHEMICALS','DEV_STYLE_NUMBER'
    if set(['MAT_NAME','PRT_NAME','PROCESS','PRT_PART_NAME','CHEMICALS','DEV_STYLE_NUMBER']).issubset(df.columns):
        df[['MAT_NAME','PRT_NAME','PROCESS','PRT_PART_NAME','CHEMICALS','DEV_STYLE_NUMBER']]= df[['MAT_NAME','PRT_NAME','PROCESS','PRT_PART_NAME','CHEMICALS','DEV_STYLE_NUMBER']].fillna('NO_DATA')
    #(Char)'PRT_COLOR'
    if 'PRT_COLOR' in df:
        df['PRT_COLOR']=df['PRT_COLOR'].fillna('NO_COLOR')
    #(int)'GRAD','TIMES'
    if 'TIMES' in df:
        df['TIMES']=df['TIMES'].fillna(0)
    if 'GRAD' in df:
        df['GRAD']=df['GRAD'].fillna(0)
    #(int)'ROUNDS'
    if 'ROUNDS' in df:
        df['ROUNDS']=df['ROUNDS'].fillna(1)

    #전부 대문자로 통일, 빈칸 지우기
    input_features=features[:4]
    df[input_features] = df[input_features].applymap(string_preprocess)

    #더이상 사용되지 않는 PROCESS 제거
    #df=df[df.PROCESS!='POP CLEAR SCREEN PRINT']
    #df=df[df.PROCESS!='PRIMER']

    #PROCESS이름이 RULE과 다른 경우 교체
    df.loc[df['PROCESS']=='A-BOND','PROCESS']='A-BOND SCREEN PRINT'
    df.loc[df['PROCESS']=='U-BOND','PROCESS']='U-BOND SCREEN PRINT'
    df.loc[df['PROCESS']=='3D CLEAR(WG-100 12%) SCREEN PRINT','PROCESS']='3D CLEAR SCREEN PRINT'
    df.loc[df['PROCESS']=='TOP GLOSSY SCREEN PRINT','PROCESS']='TOP GLOSSY CLEAR SCREEN PRINT'
    df.loc[df['PROCESS']=='HD CLEAR SCREEN PRINT','PROCESS']='HD.CLEAR SCREEN PRINT'
    df.loc[df['PROCESS']=='3D PUFF SCREEN PRINT','PROCESS']='3D CLEAR SCREEN PRINT'
    
    #str, int 섞여있는경우 str으로 통일
    df = df.astype({'DEV_STYLE_NUMBER':'str'})
    
    #index 순서대로
    df=df.reset_index(drop=True)

    return df

In [29]:
def string_preprocess(s):
    s=str(s).upper()
    s_split=s.split()
    s_split = [x for x in s_split if not str(x).isdigit()]
    s=' '.join(s_split)
    return s

In [30]:
#입력: Test할 데이터프레임과 현재 알고싶은 행의 인덱스 i
#출력: 데이터베이스에서의 위치 혹은 없을 경우 -1
def search_database(database, df,i):
    #데이터 베이스와 비교할 칼럼과 그 결과를 기록할 칼럼 생성
    compare_col=['DEV_STYLE_NUMBER','PROCESS','PRT_PART_NAME','MAT_NAME','PRT_NAME','PRT_COLOR','CHEMICALS','GRAD']
    
    #찾고싶은 행을 데이터베이스의 하단에 추가
    temp=database.append(df[compare_col].loc[i])
    if len(temp[temp.duplicated(compare_col,keep=False)])!=0:  #동일한게 있으면 
        location=temp[temp.duplicated(compare_col,keep=False)].index[0] #location은 데이터베이스에서 자신과 같은 행의 위치
        return location
    else:
        return -1

In [31]:
#Catboost를 이용한 예측
#입력: 기존에 없던 TEST 데이터프레임, dictionary
#출력: 예측을 완료한 데이터프레임
def prediction_catboost(dictionary,df):
    
    #기존에 학습한 Catboost 모델 가져오기
    Catboost_model=CatBoostClassifier(iterations=50, random_state=24)
    Catboost_model.load_model( os.path.join(OUTPUT_DATA_DIR ,'Model/catboost_TIMES_ROUNDS'),format="cbm")
    
    #dictionary의 결과로 나오도록 Catboost모델 이용하여 예측
    features=['DEV_STYLE_NUMBER','PROCESS','PRT_PART_NAME','MAT_NAME','PRT_NAME','PRT_COLOR','CHEMICALS','GRAD']
    y_pred_TIMES_ROUNDS=Catboost_model.predict(pred_set[features]).reshape(-1)
    result=list(y_pred_TIMES_ROUNDS)

    #dictionary의 결과를 각각 PRED_TIMES, PRED_ROUNDS로 옮기기
    
    df['TIMES_ROUND'] = df[['TIMES','ROUNDS']].T.apply(lambda x: dictionary[(x[0], x[1])])
    return df

In [32]:
#데이터베이스 검색을 이용한 예측
#입력: 기존 데이터베이스에 있는 TEST 데이터
def prediction_database(database, df):
    for i in list(df.index):
        database_idx=int(df['location'].loc[i]) #database index 값 찾기
        df.loc[i,'PRED_TIMES']=database.loc[database_idx,'TIMES']
        df.loc[i,'PRED_ROUNDS']=database.loc[database_idx,'ROUNDS']
    return df

In [33]:
#Test 데이터에서 Feature이같은데 TIMES, ROUNDS가 다른 데이터를 걸러내는 함수
#def Test_data_inspection(df):
#    df[df.duplicated(['DEV_STYLE_NUMBER','PROCESS','PRT_PART_NAME','MAT_NAME','PRT_NAME','PRT_COLOR','CHEMICALS','GRAD'],keep=False)]
#    df=df.drop_duplicates(['DEV_STYLE_NUMBER','PROCESS','PRT_PART_NAME','MAT_NAME','PRT_NAME','PRT_COLOR','CHEMICALS','GRAD'],keep='first')   

# Main

In [34]:
#Databse폴더에서 데이터베이스 읽어오기
xls = pd.ExcelFile(files_DATABASE[0],engine='openpyxl')
database = xls.parse('Sheet1',header=0)

#Database 전처리
database=database.fillna('None')
database=database.drop_duplicates(['DEV_STYLE_NUMBER','PROCESS','PRT_PART_NAME','MAT_NAME','PRT_NAME','PRT_COLOR','CHEMICALS','GRAD'],keep='first')

In [35]:
#Database데이터의 TIMES,ROUNDS를 한쌍으로 만들어서 Dictionary로 만들기
dictionary=TIMES_ROUNDS_dictionary(database)

In [36]:
dictionary

{0: (2, 1),
 1: (3, 3),
 2: (4, 4),
 3: (12, 6),
 4: (4, 2),
 5: (2, 2),
 6: (1, 1),
 7: (0, 1),
 8: (0, 6),
 9: (16, 8),
 10: (7, 7),
 11: (5, 5),
 12: (6, 3),
 13: (8, 4),
 14: (10, 5),
 15: (0, 4),
 16: (24, 12),
 17: (6, 6),
 18: (0, 3),
 19: (3, 6),
 20: (6, 4),
 21: (2, 4),
 22: (0, 2),
 23: (4, 3),
 24: (12, 4),
 25: (6, 12)}

In [66]:
while True:
    state=input("원하는 작업의 번호를 입력하고 Enter를 입력하세요.\n\n1.TIMES,ROUNDS 예측 \n2.모델 Update \n3.프로그램 종료 \n\n번호를 입력하세요:")
    
    if state=='1':
        #Test 데이터에 대해 예측

        print("\n")
        print("<작업로그>")
        #Test 데이터 불러오기
        filename=os.listdir(TEST_DATA_DIR)[0]
        xls = pd.ExcelFile(os.path.join(TEST_DATA_DIR,filename),engine='openpyxl')
        df = xls.parse('Sheet1',header=0)
        df.head()

        #Test 데이터 전처리
        print("Test 데이터 전처리 진행중")
        features=['DEV_STYLE_NUMBER','PROCESS','PRT_PART_NAME','MAT_NAME','PRT_NAME','PRT_COLOR','CHEMICALS','GRAD','TIMES','ROUNDS']
        df=data_preprocess(df, features)

        #PRED_TIMES, PRED_ROUNDS, location칼럼 추가
        df.insert(list(df.columns).index('TIMES'),'PRED_TIMES',-1)
        df.insert(list(df.columns).index('TIMES'),'PRED_ROUNDS',-1)
        df['location']=-1

        #데이터 베이스에 있는지 없는지 확인
        print("데이터베이스 기록 확인 진행중")
        print("이 과정은 시간이 소요됩니다. 잠시만 기다려주십시오.")
        for i in range(len(df)):
            df.loc[i,'location']=search_database(database, df,i)

        #있는건 데이터 베이스에서 찾아서
        search_set=df[df['location']!=-1]
        #search_set=prediction_database(database, search_set)
        search_set=prediction_database(dictionary, search_set)

        #없는건 Catboost로 예측해서
        pred_set=df[df['location']==-1]
        pred_set=prediction_catboost(dictionary, pred_set)

        #두개 합치기
        #location 없애기
        final=pd.concat([pred_set,search_set]).sort_index()
        final.drop(columns=['location'],axis=1,inplace=True)
        
        #최종 예측 완료 파일 엑셀로 저장
        print("예측완료 파일 저장")
        final.to_excel(os.path.join(OUTPUT_DATA_DIR,'Final.xlsx'),index=False,engine='openpyxl')      
        
        #있는데 틀린거에 대해서 데이터베이스를 업데이트 해주는 함수
        print("기존에 존재하는 데이터베이스 업데이트 중")
        temp=search_set[(search_set['PRED_TIMES']!=search_set['TIMES']) | (search_set['PRED_ROUNDS']!=search_set['ROUNDS'])]
        for i in list(temp.index):
            database_idx=temp.loc[i,'location']
            database.loc[database_idx,'TIMES']=temp.loc[i,'TIMES']
            database.loc[database_idx,'ROUNDS']=temp.loc[i,'ROUNDS']
        temp.to_excel(os.path.join(OUTPUT_DATA_DIR,'Search_update.xlsx'),index=False,engine='openpyxl')
        
        #없는데 틀린것에 대해서 데이터베이스에 추가 해주는 함수
        print("새로 추가된 데이터를 데이터베이스에 업데이트 중")
        temp=pred_set[(pred_set['PRED_TIMES']!=pred_set['TIMES']) | (pred_set['PRED_ROUNDS']!=pred_set['ROUNDS'])]
        temp=temp.drop_duplicates(features,keep='first')
        temp=temp[['DEV_STYLE_NUMBER','PROCESS','PRT_PART_NAME','MAT_NAME','PRT_NAME','PRT_COLOR','CHEMICALS','GRAD','TIMES','ROUNDS']]
        temp.to_excel(os.path.join(OUTPUT_DATA_DIR,'Predict_update.xlsx'),index=False,engine='openpyxl')
        database=pd.concat([database,temp],ignore_index=True)
        
        #데이터베이스 최종 업데이트
        database.to_excel(files_DATABASE[0],index=False,engine='openpyxl')
        
        print("TIMES,ROUNDS 예측이 완료되었습니다.")
        print("------------------------------------------")
        
    elif state=='2':
        #Database로 부터 모델 생성
        catboost_model(database, dictionary)
        
    elif state=='3':
        #프로그램 종료
        break
    else:
        print("잘못된 번호가 삽입 되었습니다. 다시 입력해주세요:")

원하는 작업의 번호를 입력하고 Enter를 입력하세요.

1.TIMES,ROUNDS 예측 
2.모델 Update 
3.프로그램 종료 

번호를 입력하세요:3
